# Обучение структуры

## Задать переменные sys

In [14]:
import os,sys
from pathlib import Path
currentdir = Path(os.getcwd())
sys.path.insert(0, os.path.dirname(currentdir.parent))
sys.path.insert(0, os.path.dirname(currentdir))
print(str(sys.path[0:3]))

['c:\\Users\\Worker1\\Documents\\BAMT', 'c:\\Users\\Worker1\\Documents', 'c:\\Users\\Worker1\\Documents\\BAMT']


## Испортировать модули

In [4]:
%%time
%matplotlib inline
import BAMT.Networks as Nets
import BAMT.Preprocessors as pp

import pandas as pd
import numpy as np
from sklearn import preprocessing
import matplotlib.pyplot as plt


CPU times: total: 4.19 s
Wall time: 11.4 s


## Предобработка

In [5]:
# Подгружаем данные

hack = pd.read_csv(r'../Data/hack_processed_with_rf.csv')

In [6]:
# Выбираем колонки, вытаскиваем их

cols = ['Tectonic regime', 'Period', 'Lithology', 'Structural setting', 'Gross','Netpay','Porosity','Permeability', 'Depth']
hack = hack[cols]

In [7]:
# encoder - для категориальных (обозначает их через 0, 1, 2, ...)
# discretizer - для непрерывных (дискретизируем)
    # n_bins - разделить непрерывные данные на интервалы по n_bins
    # encode: метод кодирования. 'ordinal' - кодирование целыми числами
    # strategy: стратегия для определения ширины инетрвала. 'quantile' - все интервалы имеют одинаковое количество значений

encoder = preprocessing.LabelEncoder()
discretizer = preprocessing.KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='quantile')

# discretized_data - закодированные данные
# est - словарь кодирования для категориальных признаков
# p - кодирующий объект
p = pp.Preprocessor([('encoder', encoder), ('discretizer', discretizer)])
discretized_data, est = p.apply(hack)

## Инициализация байесовской сети

<p> Существует 3 типа байесовских сетей - DiscreteBN, ContinuousBN, HybridBN (Дискретные, непрерывные и гибридные). <br>
Обратите внимание, что если вы передаете дискретные данные в Continoust BN, вы получаете ошибку.<br><br>
Для ContinousBN пользователь может выбрать, использовать ли смешанные узлы или нет, для гибридного пользователь может ограничить/разрешить использование логит/смешанных узлов.<p> 

Список scoring_functions, с которыми может работать BAMT: <br>
1. Group 1
    - Mutual Information (MI)
    - LL
    - BIC
    - AIC
2. K2Score <br><br>
<p> Для group 1 пользователь может передать кортеж как ('MI',). Для K2Score пользователь должен импортировать K2Scorer и передать ("K2", K2Scorer)<p>

In [8]:
# has_logit - делать ли связи от непрерывных к дискретным (будет модель классификции: логист. регрес.)
# use_mixture - использовать ли смеси (лучше опишет)
# signs - знак положительный или отрицательный (нужно для семплирования, что рост вдруг не оказался отрицательным)


bn = Nets.HybridBN(has_logit=True, use_mixture=True) # создание BN
info = p.info # отображение узлов (Dict["types": Dict[node_name: type], "signs": Dict[node_name: sign]])

In [9]:
info

{'types': {'Tectonic regime': 'disc',
  'Period': 'disc',
  'Lithology': 'disc',
  'Structural setting': 'disc',
  'Gross': 'cont',
  'Netpay': 'cont',
  'Porosity': 'cont',
  'Permeability': 'cont',
  'Depth': 'cont'},
 'signs': {'Gross': 'pos',
  'Netpay': 'pos',
  'Porosity': 'pos',
  'Permeability': 'pos',
  'Depth': 'pos'}}

Структурное обучение состоит из двух частей: построения узлов и построения ребер. <br><br>

Первая стадия: <br>
Экземпляр байесовской сети инициализировал первичные узлы внутри двумя типами ('Discrete' и 'Gaussian') в соответствии с информацией дескриптора.<br>
Вторая стадия: <br>
Экземпляр байесовской сети перезаписывает узлы в соответствии с их родителями и условиями (параметрами)

In [10]:
bn.add_nodes(info)
# ТАМ ДОЛЖНЫ БЫТЬ ПРЕДВАРИТЕЛЬНО ОБРАБОТАННЫЕ ДАННЫЕ 
bn.add_edges(discretized_data, scoring_function=('MI',)) # использовать score function mutual information, реализованную в BAMT

In [11]:
bn.get_info()

,name,node_type,data_type,parents,parents_types
0,Tectonic regime,Discrete,disc,[],[]
1,Period,ConditionalLogit (LogisticRegression),disc,"[Depth, Structural setting, Lithology]","[cont, disc, disc]"
2,Lithology,ConditionalLogit (LogisticRegression),disc,"[Netpay, Structural setting]","[cont, disc]"
3,Structural setting,Logit (LogisticRegression),disc,[Permeability],[cont]
4,Gross,MixtureGaussian,cont,[Porosity],[cont]
5,Netpay,MixtureGaussian,cont,[Permeability],[cont]
6,Porosity,ConditionalMixtureGaussian,cont,[Tectonic regime],[disc]
7,Permeability,MixtureGaussian,cont,[Gross],[cont]
8,Depth,MixtureGaussian,cont,[Gross],[cont]


In [12]:
# Строим граф. В кавычках название
bn.plot('Test1')
# надо указывать расширение html

2022-03-05 14:16:21,126 | ERROR    | Networks.py-plot-0343 | This version allows only html format.


In [13]:
# Теперь построим граф
bn.plot('Simple.html')
# Цвета рандомные